In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time
from webdriver_manager.chrome import ChromeDriverManager
import sqlite3
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import re

In [3]:
csv = pd.read_csv("./Final_Data_add_etc.csv")
print(csv.columns)

Index(['YM', 'MCT_NM', 'OP_YMD', 'MCT_TYPE', 'ADDR', 'UE_CNT_GRP',
       'UE_AMT_GRP', 'UE_AMT_PER_TRSN_GRP', 'MON_UE_CNT_RAT', 'TUE_UE_CNT_RAT',
       'WED_UE_CNT_RAT', 'THU_UE_CNT_RAT', 'FRI_UE_CNT_RAT', 'SAT_UE_CNT_RAT',
       'SUN_UE_CNT_RAT', 'HR_5_11_UE_CNT_RAT', 'HR_12_13_UE_CNT_RAT',
       'HR_14_17_UE_CNT_RAT', 'HR_18_22_UE_CNT_RAT', 'HR_23_4_UE_CNT_RAT',
       'LOCAL_UE_CNT_RAT', 'RC_M12_MAL_CUS_CNT_RAT', 'RC_M12_FME_CUS_CNT_RAT',
       'RC_M12_AGE_UND_20_CUS_CNT_RAT', 'RC_M12_AGE_30_CUS_CNT_RAT',
       'RC_M12_AGE_40_CUS_CNT_RAT', 'RC_M12_AGE_50_CUS_CNT_RAT',
       'RC_M12_AGE_OVR_60_CUS_CNT_RAT', 'key', 'MCT_NAVER_NAME', 'placeID',
       '휴업일', '영업시간', '전화번호', '메뉴', 'ADDR_filtered', '시설정보', '주차정보', '좌석',
       '결제', '분류', 'review'],
      dtype='object')


In [21]:

csv = pd.read_csv("./Final_Data_add_etc.csv")


missing_placeID = csv[csv['placeID'].isna()][['key', 'MCT_NM', 'ADDR']]

missing_placeID['MCT_NM'] = missing_placeID['MCT_NM'].apply(
    lambda x: re.sub(r'\s*\(.*?\)\s*', '', x))

missing_placeID['ADDR'] = missing_placeID['ADDR'].apply(
    lambda x: ' '.join(x.split()[1:3]) if len(x.split()) > 2 else x)

missing_placeID['Combined'] = missing_placeID['MCT_NM'] + \
    ' ' + missing_placeID['ADDR']


conn = sqlite3.connect('./place.db')
cursor = conn.cursor()


existing_keys = cursor.execute('SELECT key FROM restaurant').fetchall()
existing_keys = {key[0] for key in existing_keys}

missing_placeID = missing_placeID[~missing_placeID['key'].isin(existing_keys)]

chrome_options = Options()
prefs = {
    "profile.managed_default_content_settings.images": 2
}
chrome_options.add_experimental_option("prefs", prefs)


proxy = "45.142.153.210:50100"
chrome_options.add_argument(
    f"--proxy-server={proxy}")

proxy_id = "l5405018481039679"
proxy_pw = "g2Je24mLpI"


driver = webdriver.Chrome(options=chrome_options)
driver.get("https://map.naver.com/")

time.sleep(10)

total_rows = len(missing_placeID)

for index, row in missing_placeID.iterrows():
    start_time = time.time()
    i = row['key']
    name = row['Combined']
    if i % 2 == 0:
        continue

    driver.get(f"https://map.naver.com/p/search/{name}")

    print(f"\ngood \t | searching...... waiting 3s | [{i}] {name}")
    try:
        iframe = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.ID, 'entryIframe'))
        )
        driver.switch_to.frame(iframe)
        print("good \t | iframe 안에 들어왔어요")
        try:
            url = driver.current_url
            print(f"good \t | url 가져왔어요")

            match = re.search(r'/place/(\d+)', url)

            if match:
                place_id = match.group(1)

                cursor.execute(
                    'INSERT OR REPLACE INTO restaurant (key, placeID) VALUES (?, ?)', (i, place_id))
                conn.commit()
                print(f"good \t | placeId: {place_id}   commited!")

            else:
                print(f"err \t | placeId만을 뽑아내지 못했어요... 왜지.. {url}")

        except:
            print("!pass\t | 해당 이름이 여러 개 있나봐요..")
    except:
        print("err \t | iframe 안에 들어오지 못했어요... 하나도 없나봐요...")
        cursor.execute(
            'INSERT OR REPLACE INTO restaurant (key, placeID) VALUES (?, ?)', (i, None))
        conn.commit()

    end_time = time.time()

    print(
        f"!TIME! \t | {index + 1}/{total_rows} \t | {end_time-start_time}")

conn.close()
driver.quit()

In [32]:

csv = pd.read_csv("./Final_Data_add_etc.csv")


missing_placeID = csv[csv['placeID'].isna()][['key', 'MCT_NM', 'ADDR']]

missing_placeID['MCT_NM'] = missing_placeID['MCT_NM'].apply(
    lambda x: re.sub(r'\s*\(.*?\)\s*', '', x))

missing_placeID['ADDR'] = missing_placeID['ADDR'].apply(
    lambda x: ' '.join(x.split()[1:3]) if len(x.split()) > 2 else x)

missing_placeID['Combined'] = missing_placeID['MCT_NM'] + \
    ' ' + missing_placeID['ADDR']


conn = sqlite3.connect('./place.db')
cursor = conn.cursor()


existing_place_ids = cursor.execute(
    'SELECT key FROM restaurant WHERE placeID IS NOT NULL').fetchall()
existing_place_ids = {key[0] for key in existing_place_ids}

missing_placeID = missing_placeID[~missing_placeID['key'].isin(
    existing_place_ids)]

# print(missing_placeID)

for index, row in missing_placeID.iterrows():
    start_time = time.time()
    i = row['key']
    original_name = row['MCT_NM']
    original_address = row['ADDR']

    print(i, original_name, original_address)

    cursor.execute(
        'UPDATE restaurant SET original_name = ?, original_address = ? WHERE key = ?', (
            original_name, original_address, i)
    )
conn.commit()

1 한국수상레저안전협회 제주제주시지부 제주시 이호일동
3 케이디에셋 담앤루 서귀포시 대포동
5 비케이알 버거킹 제주화북DT점 제주시 화북일동
6 성우디엔에프 서귀포시 강정동
7 신세계푸드 제주신화월드  고래라면 서귀포시 안덕면
11 아워홈 제주대학교병원제주점 제주시 아라일동
13 아워홈 제주한라병원점 제주시 연동
14 현대그린푸드 국세공무원교육원 서귀포시 서호동
15 구푸디엉또정 서귀포시 강정동
16 그라스랜드 제주시 애월읍
18 네츄럴파크리조트 제주시 애월읍
19 다모아미래컨벤션센터 제주시 용담일동
20 리앤이라마띠네제주호텔브릿지서귀포점 서귀포시 서귀동
24 블루밍그릴 서귀포시 색달동
25 비케이알 버거킹 제주노형점 제주시 노형동
28 새들러컴퍼니 제주지점 제주시 애월읍
30 서울인더스트리포트그릴스테이크하우스 서귀포시 서귀동
31 시더스초밥제주연동점 제주시 연동
32 신세계푸드 세인트존스베리 아카데미 서귀포시 대정읍
33 신세계푸드 제주신화월드 서귀포시 안덕면
34 신세계푸드 제주신화월드 직원식당 서귀포시 안덕면
35 신세계푸드 한국공항공사 제주지역본부 직원식당 제주시 용담이동
36 아워홈 네오플제주점 제주시 노형동
37 아워홈 렛츠런파크제주점 제주시 애월읍
38 아워홈 렛츠런파크제주점 한식당 제주시 애월읍
39 아워홈 제주국제공항2지점 제주시 용담이동
41 아워홈 제주지방합동청사제주점 제주시 도남동
42 아워홈 카카오제주점 제주시 영평동
43 에스피오엘  제주지점_한식 제주시 애월읍
44 엠브릿지 서귀포시 대정읍
47 오제이상사애월지점 낭푼밥상 제주시 연동
48 옹포바다횟집 제주시 한림읍
50 제담에프앤비 제주호랭이 성산점 서귀포시 성산읍
51 제이콥에프앤비 제주아일랜드점 서귀포시 안덕면
53 제주테마조각공원 제주시 연동
54 젠하이더웨이묵전제주점 서귀포시 대정읍
55 지이오코리아 제주시 노형동
58 캐롯닷밀 제주시 애월읍
60 케이에프씨코리아 KFC서귀포DI 서귀포시 동홍동
61 코드넷 제주시 오라이동
64 파리크라상파스쿠찌제주대병원 제주시 아라일동
66

In [16]:

csv = pd.read_csv("./Final_Data_add_etc.csv")


missing_placeID = csv[csv['placeID'].isna(
)][['key', 'MCT_NM', 'ADDR']]

missing_placeID['MCT_NM'] = missing_placeID['MCT_NM'].apply(
    lambda x: re.sub(r'\s*\(.*?\)\s*', '', x))

missing_placeID['ADDR'] = missing_placeID['ADDR']

missing_placeID['Combined'] = missing_placeID['MCT_NM'] + \
    ' ' + missing_placeID['ADDR'].apply(
    lambda x: ' '.join(x.split()[1:3]) if len(x.split()) > 2 else x)


conn = sqlite3.connect('./place.db')
cursor = conn.cursor()


existing_place_ids = cursor.execute(
    'SELECT key FROM restaurant WHERE placeID IS NOT NULL AND error_code IS NOT NULL').fetchall()
existing_place_ids = {key[0] for key in existing_place_ids}


missing_placeID = missing_placeID[~missing_placeID['key'].isin(
    existing_place_ids)]

chrome_options = Options()
prefs = {
    "profile.managed_default_content_settings.images": 2
}
chrome_options.add_experimental_option("prefs", prefs)


proxy = "45.142.153.210:50100"
# chrome_options.add_argument(
#     f"--proxy-server={proxy}")

proxy_id = "l5405018481039679"
proxy_pw = "g2Je24mLpI"


driver = webdriver.Chrome(options=chrome_options)
driver.get("https://map.naver.com/")


total_rows = len(missing_placeID)

for index, row in missing_placeID.iterrows():
    start_time = time.time()
    i = row['key']
    combined_name = row['Combined']
    addr = row['ADDR']
    # if i % 2 == 0:
    #     continue

    try:
        driver.get(f"https://map.naver.com/p/search/{combined_name}")
        print(
            f"\ngood \t | COMBINED_NAME searching...... waiting 3s | [{i}] {combined_name}")

        iframe = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.ID, 'searchIframe'))
        )
        driver.switch_to.frame(iframe)
        print("good \t | iframe 안에 들어왔어요..")
        try:
            restaurant = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable(
                    (By.CLASS_NAME, 'place_bluelink'))
            )
            restaurant.click()
            print("good \t | 매장?을 클릭했어요.")
            entry_iframe = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.ID, 'entryIframe'))
            )
            driver.switch_to.frame(entry_iframe)
            print("good \t | entryIframe 안에 들어왔어요.")

            naver_name = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'GHAhO'))
            ).text
            naver_address = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'LDgIH'))
            ).text
            cursor.execute(
                'UPDATE restaurant SET naver_name = ?, naver_address = ? WHERE key = ?', (
                    naver_name, naver_address, i)
            )
            conn.commit()
            print(
                f"success! \t | 네이버에서 가져온 '{naver_name}' | '{naver_address}'를 commit 했어요.")

        except:
            print("nope \t | 없나봐요 iframe 나올게요.")

        print(0)
        driver.switch_to.default_content()
        driver.get(f"https://map.naver.com/p/search/{addr}")

        print(1)
        print(
            f"\ngood \t | FULL ADDR searching...... waiting 3s | [{i}] {addr}")
        try:
            print(2, "in try")
            try:
                search_result_title = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located(
                        (By.CLASS_NAME, 'search_result_title'))
                )
                print(3)
                search_result_title.click()
                print("good \t | 가장 처음 나온 주소?를 클릭했어요.")

                link_space = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located(
                        (By.CLASS_NAME, 'link_space'))
                )
                link_space.click()
                print("good \t | 가장 처음 나온 곳을 클릭했어요.")
            except:
                try:
                    link_space = WebDriverWait(driver, 1).until(
                        EC.presence_of_element_located(
                            (By.CLASS_NAME, 'link_space'))
                    )
                    print(4)
                    link_space.click()
                    print("클릭했어요")
                except:
                    cursor.execute(
                        'UPDATE restaurant SET error_code = ? WHERE key = ?', (
                            "정확한 주소가 아녜요", i)
                    )
                    print("error code 커밋했어요")
                    continue

            entry_iframe = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.ID, 'entryIframe'))
            )
            driver.switch_to.frame(entry_iframe)
            print("good \t | entryIframe 안에 들어왔어요.")

            naver_name = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'GHAhO'))
            ).text
            naver_address = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'LDgIH'))
            ).text
            cursor.execute(
                'UPDATE restaurant SET naver_name = ?, naver_address = ?, error_code = ? WHERE key = ?', (
                    naver_name, naver_address, "", i)
            )
            conn.commit()
            print(
                f"success! \t | 네이버에서 가져온 '{naver_name}' | '{naver_address}'를 commit 했어요.")

        except Exception as e:
            print(f"err \t | {e}")

    except:
        print("what")
        continue

conn.close()
driver.quit()


good 	 | COMBINED_NAME searching...... waiting 3s | [1] 한국수상레저안전협회 제주제주시지부 제주시 이호일동
good 	 | iframe 안에 들어왔어요..
nope 	 | 없나봐요 iframe 나올게요.
0
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what